# Process and feature engineering on the logistics dataset

1. Load data
2. Create an out-of-stock binary variable
3. Create window calculations of the last value and rolling averages

In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
%matplotlib inline

In [3]:
# Primary Dataset
lc_data = pd.read_csv("final_data/contraceptive_logistics_data.csv")
lcd_date_dict = dict(year = lc_data.year,
                     month = lc_data.month,
                     day = [28]*len(lc_data))
lc_data['year_month']  = pd.to_datetime(lcd_date_dict)
lc_data.to_csv("lc_data.csv",index=False)
lc_data.head()

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,year_month
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,75,0,21,-54,0,18,0,100.0,2019-01-28
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3,0,3,0,0,2,0,10.0,2019-01-28
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,0,75,22,0,53,9,0,0.0,2019-01-28
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,2,0,0,0,2,0,0,0.0,2019-01-28
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,45,0,2,0,43,1,0,0.0,2019-01-28


## Feature Engineering

__Potential__
- Lagging indicator
- Out of stock variable
- Clustering label for products
- Clustering label for sites

__Notes__
- timeseries data is not symmetrical
- We need to write our own algorithm to handle the calculation of lagging indicators

### Stockout during period

In [4]:
# Compute an out of stock variable
have_stock = lc_data["stock_stockout_days"] == 0 
stock_event = []
for x in have_stock:
    stock_event.append(int(x == False))
lc_data["out_of_stock_during_period"] = stock_event

In [5]:
lc_data['stock_change'] = lc_data['stock_initial'] - lc_data['stock_end']

In [6]:
lc_data['stock_pct_change'] = (lc_data['stock_initial'] - lc_data['stock_end'])/ lc_data['stock_initial']

In [7]:
lc_data.head()

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,year_month,out_of_stock_during_period,stock_change,stock_pct_change
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,75,0,21,-54,0,18,0,100.0,2019-01-28,0,75,1.000000
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3,0,3,0,0,2,0,10.0,2019-01-28,0,3,1.000000
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,0,75,22,0,53,9,0,0.0,2019-01-28,0,-53,-inf
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,2,0,0,0,2,0,0,0.0,2019-01-28,0,0,0.000000
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,45,0,2,0,43,1,0,0.0,2019-01-28,0,2,0.044444


## Calculate timeseries indicators for each site and product

- Calculate last value
- Calculate last value 12 months ago
- Calculate average value for the last 3 periods
- Calculate average value for the last 6 periods
- Calculate average value for the last 12 periods
- Calculate min value last 3 periods
- Calculate min value last 6 periods
- Calculate min value last 12 periods
- Calculate max value last 3 periods
- Calculate max value last 6 periods
- Calculate max value last 12 periods

In [8]:
numeric_cols = ['stock_initial',
                'stock_received',
                'stock_distributed',
                'stock_ordered',
                'stock_adjustment',
                'stock_end',
                'average_monthly_consumption']
lc_data_grouped = lc_data.set_index(['site_code','product_code'])

In [9]:
test_df = lc_data_grouped.loc[('C1024','AS21126'),:].sort_values(by='year_month').set_index('year_month',append=True)

#test_df

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [10]:
# Moves values one step into the series
test_df['stock_distributed'].shift(1)


site_code  product_code  year_month
C1024      AS21126       2018-12-28    NaN
                         2019-01-28    0.0
                         2019-02-28    0.0
                         2019-03-28    0.0
                         2019-04-28    0.0
                         2019-05-28    0.0
Name: stock_distributed, dtype: float64

In [11]:
# Moves Values one step backward in the series
test_df['stock_distributed'].shift(-1)

site_code  product_code  year_month
C1024      AS21126       2018-12-28    0.0
                         2019-01-28    0.0
                         2019-02-28    0.0
                         2019-03-28    0.0
                         2019-04-28    0.0
                         2019-05-28    NaN
Name: stock_distributed, dtype: float64

In [12]:
# Takes the rolling average of one value
test_df['stock_distributed'].rolling(3).mean().shift(1)

site_code  product_code  year_month
C1024      AS21126       2018-12-28    NaN
                         2019-01-28    NaN
                         2019-02-28    NaN
                         2019-03-28    0.0
                         2019-04-28    0.0
                         2019-05-28    0.0
Name: stock_distributed, dtype: float64

In [13]:
site_products = lc_data[['site_code','product_code']].drop_duplicates()
timeseries_data = {
    'lv':[],
    'lv_1yr':[],
    "avg_3":[],
    "avg_6":[],
    'avg_12':[],
    'min_3':[],
    'min_6':[],
    "min_12":[],
    "max_3":[],
    "max_6":[],
    "max_12":[]}

numeric_cols = ['stock_initial','stock_received','stock_distributed','stock_adjustment','stock_end','average_monthly_consumption']
lc_data_grouped = lc_data.set_index(['site_code','product_code'])

for index, data in site_products.iterrows():
    # Filter data to site, product pair
    site_key = data['site_code']
    product_key = data['product_code']
    
    site_product_ts = lc_data_grouped.loc[(site_key,product_key),:].sort_values(by='year_month').set_index('year_month',append=True)
    #Calculate last value
    lv = site_product_ts[numeric_cols].shift(-1)
    lv.columns = [ col+ "_lv" for col in numeric_cols]
    timeseries_data['lv'].append(lv)
    
    #Calculate last value 12 months ago
    lv_1yr = site_product_ts[numeric_cols].shift(-12)
    lv_1yr.columns = [ col+ "_lv_1yr" for col in numeric_cols]
    timeseries_data['lv_1yr'].append(lv_1yr)
    
    #Calculate average value for the last 3 periods
    avg_3 = site_product_ts[numeric_cols].rolling(3).mean().shift(1)
    avg_3.columns = [ col+ "_avg3" for col in numeric_cols]
    timeseries_data['avg_3'].append(avg_3)
    
    #Calculate average value for the last 6 periods
    avg_6 = site_product_ts[numeric_cols].rolling(6).mean().shift(1)
    avg_6.columns = [ col+ "_avg6" for col in numeric_cols]
    timeseries_data['avg_6'].append(avg_6)
    
    #Calculate average value for the last 12 periods
    avg_12 = site_product_ts[numeric_cols].rolling(12).mean().shift(1)
    avg_12.columns = [ col+ "_avg12" for col in numeric_cols]
    timeseries_data['avg_12'].append(avg_12)
    
    #Calculate min value last 3 periods
    min_3 = site_product_ts[numeric_cols].rolling(3).min().shift(1)
    min_3.columns = [ col+ "_min3" for col in numeric_cols]
    timeseries_data['min_3'].append(min_3)
    
    #Calculate min value last 6 periods
    min_6 =site_product_ts[numeric_cols].rolling(6).min().shift(1)
    min_6.columns = [ col+ "_min6" for col in numeric_cols]
    timeseries_data['min_6'].append(min_6)
    
    #Calculate min value last 12 periods
    min_12 = site_product_ts[numeric_cols].rolling(12).min().shift(1)
    min_12.columns = [ col+ "_min12" for col in numeric_cols]
    timeseries_data['min_12'].append(min_12)
    
    #Calculate max value last 3 periods
    max_3 = site_product_ts[numeric_cols].rolling(3).max().shift(1)
    max_3.columns = [ col+ "_max3" for col in numeric_cols]
    timeseries_data['max_3'].append(max_3)
    
    #Calculate max value last 6 periods
    max_6 =site_product_ts[numeric_cols].rolling(6).max().shift(1)
    max_6.columns = [ col+ "_max6" for col in numeric_cols]
    timeseries_data['max_6'].append(max_6)
    
    #Calculate max value last 12 periods
    max_12 = site_product_ts[numeric_cols].rolling(12).max().shift(1)
    max_12.columns = [ col+ "_max12" for col in numeric_cols]
    timeseries_data['max_12'].append(max_12)

In [14]:
# Add timeseries step to map as index
analytical_ts = {key:pd.concat(timeseries_data[key]) for key in timeseries_data.keys() }

# Add timeseries step data for merging
merged_analaytics_ts_df =pd.concat(analytical_ts.values(),axis = 1)
print("Shape of the generated analytics features")
print(merged_analaytics_ts_df.shape)

# Merging the transformed features onto the original data
analytical_lc_data = lc_data.set_index(["site_code",'product_code','year_month'])
analytical_lc_data = analytical_lc_data.merge(merged_analaytics_ts_df,left_index=True,right_index=True)

print("Shape of the analytics dataset after transformations")
print(analytical_lc_data.shape)

analytical_lc_data.reset_index().to_csv("analytical_data/analytical_logistics_data.csv",index=False)

Shape of the generated analytics features
(38842, 66)
Shape of the analytics dataset after transformations
(38842, 81)
